# Ch 6: Data Loading, Storage, and File Formats

In [1]:
import pandas as pd
import sys
import numpy as np
import csv

## 6.1 Reading and Writing Data in Text Format

In [3]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [6]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Comma-delimited; can use **read_csv**:

In [9]:
df = pd.read_csv('ex1.csv')
df

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

No header:

In [10]:
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [11]:
pd.read_csv('ex2.csv', header=None)

0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo

In [12]:
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

Make the 'message' column the index:

In [13]:
names=['a', 'b', 'c', 'd', 'message']

In [14]:
pd.read_csv('ex2.csv', names=names, index_col='message')

a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12

Pass a list of column numbers/names to form a heirarchical index from multiple columns:

In [16]:
!cat csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Sometimes, a table might not have a fixed delimiter:

In [19]:
list(open('ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

Notice the fields are separated by variable amounts of whitespace.  
Can pass a regular expression as a delimiter: \\s+

In [20]:
result = pd.read_csv('ex3.txt', sep='\s+')
result

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [21]:
!cat ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Skip the 1st, 3rd, and 4th rows using `skiprows`:

In [28]:
pd.read_csv('ex4.csv', skiprows=[0, 2, 3], index_col='message')

a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12

Handle missing values:

In [29]:
!cat ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [30]:
result = pd.read_csv('ex5.csv')
result

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

In [31]:
pd.isnull(result)

something      a      b      c      d  message
0      False  False  False  False  False     True
1      False  False  False   True  False    False
2      False  False  False  False  False    False

In [32]:
result = pd.read_csv('ex5.csv', na_values=['NULL'])
result

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

Can specify different NA sentinels for each column in a dict:

In [33]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [34]:
pd.read_csv('ex5.csv', na_values=sentinels)

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN

### Reading Text Files in Pieces

First, let's make the pandas display settings more compact:

In [35]:
pd.options.display.max_rows = 10

In [36]:
result = pd.read_csv('ex6.csv')
result

one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]

Use `nrows` to read a small number of rows:

In [37]:
pd.read_csv('ex6.csv', nrows=5)

one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q

Read in pieces using `chunksize`:

In [38]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

Iterate over the file, aggregating the value counts in the 'key' column:

In [41]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

tot = pd.Series([], dtype=float)
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [42]:
tot

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
     ...  
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
Length: 36, dtype: float64

In [43]:
tot = tot.sort_values(ascending=False)

In [44]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

### Writing Data to Text Format

In [45]:
data = pd.read_csv('ex5.csv')
data

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

Use `to_csv` to write to a csv file:

In [46]:
data.to_csv('out.csv')

In [47]:
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used:

In [48]:
import sys

In [49]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Can specify sentinel value using `na_rep`:

In [50]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


Can also disable row and column labels by setting `index` and `header` to **False**:

In [51]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Or, just write a subset of the columns:

In [52]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Can also use `to_csv` on a **Series**:

In [54]:
import numpy as np

In [55]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('tseries.csv')

In [56]:
!cat tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats

In [57]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


Can use Python's built-in **csv** module with single-character delimiters:

In [58]:
import csv

In [79]:
f = open('ex7.csv')
reader = csv.reader(f)

In [62]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


First, read the file into a list of lines:

In [75]:
with open('ex7.csv') as f:
    lines = list(csv.reader(f))

Split the lines into header and data lines:

In [64]:
header, values = lines[0], lines[1:]

Create a dictionary of data columns using dictionary comprehension and `zip(*values)` expression, which transposes rows to columns:

In [71]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [72]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Define a subclass of `csv.Dialect`:

In [81]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

# reader = csv.reader(f, dialect=my_dialect)

In [80]:
type(f)

_io.TextIOWrapper

In [82]:
# reader = csv.reader(f, delimiter-'|')

Can also write delimited files manually using `csv.writer` (similar to `csv.reader`):

In [83]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON Data

In [84]:
obj = """
{
 "name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

AttributeError: 'str' object has no attribute 'prettify'